## **Project Overview**

Perpustakaan Daerah Provinsi Kalimantan Selatan merupakan salah satu
perpustakaan dan pusat penyedia layanan informasi yang ada di Kalimantan
Selatan. Namun. selama ini pengunjung perpustakaan kesulitan dalam mencari
buku yang berkaitan dengan buku yang dipilih sebelumnya dan juga dalam
menemukan alternatif buku lain ketika buku yang diinginkan tersebut telah
dipinjam. dengan adanya rekomendasi atau saran buku-buku lain yang
berhubungan diharapkan membantu dalam mendapatkan buku yang sesuai dan
diinginkan pengunjung perpustakaan. Pada penelitian ini penerapan sistem
rekomendasi menggunakan metode Content Based Filtering dalam
memberikan rekomendasi buku yang bekerja dengan melihat kemiripan item
yang dianalisis dari fitur yang dikandungnya dengan Weighted Tree Similarity.
Berdasarkan hasil pengujian yang telah dilakukan pada 5 skenario pengujian
yang diujikan dihasilkan nilai precision sebesar 88%.

referensi dari proyek overview yang saya buat dapat dilihat dari tautan berikut : [Jurnal Manajemen, Teknik Informatika dan Rekayasa Komputer](https://journal.universitasbumigora.ac.id/index.php/matrik/article/download/617/587/)

Datasets : Best Books (10k) Multi-Genre Data

Source : Kaggle

Link : [Best Books (10k) Multi-Genre Data](https://www.kaggle.com/datasets/ishikajohari/best-books-10k-multi-genre-data)

# **Data Understanding**



In [7]:
# Install opendataset
!pip install opendatasets

In [8]:
# import all needed libraries
import pandas as pd
import opendatasets as od

In [9]:
# Upload kaggle.json (jalankan ini dan upload file-nya saat diminta)
from google.colab import files
files.upload()


Saving kaggle.json to kaggle (1).json


{'kaggle (1).json': b'{"username":"astars27","key":"2e92fada7b6d8479b49c78715caa705a"}'}

In [10]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [11]:
# Remove the existing folder if it exists
!rm -rf best-books-10k-multi-genre-data
# Download the dataset using the Kaggle API
!kaggle datasets download -d ishikajohari/best-books-10k-multi-genre-data
# Extract the downloaded zip file
!unzip best-books-10k-multi-genre-data.zip -d best-books-10k-multi-genre-data

Dataset URL: https://www.kaggle.com/datasets/ishikajohari/best-books-10k-multi-genre-data
License(s): CC0-1.0
best-books-10k-multi-genre-data.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  best-books-10k-multi-genre-data.zip
  inflating: best-books-10k-multi-genre-data/goodreads_data.csv  


In [12]:
books = pd.read_csv('/content/best-books-10k-multi-genre-data/goodreads_data.csv')
books

,Unnamed: 0,Book,Author,Description,Genres,Avg_Rating,Num_Ratings,URL
0,0,To Kill a Mockingbird,Harper Lee,The unforgettable novel of a childhood in a sl...,"['Classics', 'Fiction', 'Historical Fiction', ...",4.27,"5,691,311",https://www.goodreads.com/book/show/2657.To_Ki...
1,1,Harry Potter and the Philosopher’s Stone (Harr...,J.K. Rowling,Harry Potter thinks he is an ordinary boy - un...,"['Fantasy', 'Fiction', 'Young Adult', 'Magic',...",4.47,"9,278,135",https://www.goodreads.com/book/show/72193.Harr...
2,2,Pride and Prejudice,Jane Austen,"Since its immediate success in 1813, Pride and...","['Classics', 'Fiction', 'Romance', 'Historical...",4.28,"3,944,155",https://www.goodreads.com/book/show/1885.Pride...
3,3,The Diary of a Young Girl,Anne Frank,Discovered in the attic in which she spent the...,"['Classics', 'Nonfiction', 'History', 'Biograp...",4.18,"3,488,438",https://www.goodreads.com/book/show/48855.The_...
4,4,Animal Farm,George Orwell,Librarian's note: There is an Alternate Cover ...,"['Classics', 'Fiction', 'Dystopia', 'Fantasy',...",3.98,"3,575,172",https://www.goodreads.com/book/show/170448.Ani...
...,...,...,...,...,...,...,...,...
9995,9995,"Breeders (Breeders Trilogy, #1)",Ashley Quigley,How far would you go? If human society was gen...,"['Dystopia', 'Science Fiction', 'Post Apocalyp...",3.44,276,https://www.goodreads.com/book/show/22085400-b...
9996,9996,Dynamo,Eleanor Gustafson,Jeth Cavanaugh is searching for a new life alo...,[],4.23,60,https://www.goodreads.com/book/show/20862902-d...
9997,9997,The Republic of Trees,Sam Taylor,This dark fable tells the story of four Englis...,"['Fiction', 'Horror', 'Dystopia', 'Coming Of A...",3.29,383,https://www.goodreads.com/book/show/891262.The...
9998,9998,"Waking Up (Healing Hearts, #1)",Renee Dyer,For Adriana Monroe life couldn’t get any bette...,"['New Adult', 'Romance', 'Contemporary Romance...",4.13,263,https://www.goodreads.com/book/show/19347252-w...


In [13]:
print('Books data numbers : ', len(books.Book.unique()))

Books data numbers :  9871


In [14]:
# Drop Unnamed: 0	column
books = books.drop('Unnamed: 0', axis=1)
books

,Book,Author,Description,Genres,Avg_Rating,Num_Ratings,URL
0,To Kill a Mockingbird,Harper Lee,The unforgettable novel of a childhood in a sl...,"['Classics', 'Fiction', 'Historical Fiction', ...",4.27,"5,691,311",https://www.goodreads.com/book/show/2657.To_Ki...
1,Harry Potter and the Philosopher’s Stone (Harr...,J.K. Rowling,Harry Potter thinks he is an ordinary boy - un...,"['Fantasy', 'Fiction', 'Young Adult', 'Magic',...",4.47,"9,278,135",https://www.goodreads.com/book/show/72193.Harr...
2,Pride and Prejudice,Jane Austen,"Since its immediate success in 1813, Pride and...","['Classics', 'Fiction', 'Romance', 'Historical...",4.28,"3,944,155",https://www.goodreads.com/book/show/1885.Pride...
3,The Diary of a Young Girl,Anne Frank,Discovered in the attic in which she spent the...,"['Classics', 'Nonfiction', 'History', 'Biograp...",4.18,"3,488,438",https://www.goodreads.com/book/show/48855.The_...
4,Animal Farm,George Orwell,Librarian's note: There is an Alternate Cover ...,"['Classics', 'Fiction', 'Dystopia', 'Fantasy',...",3.98,"3,575,172",https://www.goodreads.com/book/show/170448.Ani...
...,...,...,...,...,...,...,...
9995,"Breeders (Breeders Trilogy, #1)",Ashley Quigley,How far would you go? If human society was gen...,"['Dystopia', 'Science Fiction', 'Post Apocalyp...",3.44,276,https://www.goodreads.com/book/show/22085400-b...
9996,Dynamo,Eleanor Gustafson,Jeth Cavanaugh is searching for a new life alo...,[],4.23,60,https://www.goodreads.com/book/show/20862902-d...
9997,The Republic of Trees,Sam Taylor,This dark fable tells the story of four Englis...,"['Fiction', 'Horror', 'Dystopia', 'Coming Of A...",3.29,383,https://www.goodreads.com/book/show/891262.The...
9998,"Waking Up (Healing Hearts, #1)",Renee Dyer,For Adriana Monroe life couldn’t get any bette...,"['New Adult', 'Romance', 'Contemporary Romance...",4.13,263,https://www.goodreads.com/book/show/19347252-w...


## **Univariate Exploratory Data Analysis**

Variabel-variabel pada dataset Best Books (10k) Multi-Genre Data adalah sebagai berikut :

- Book: merupakan judul buku
- Author:  merupakan pengarang buku
- Description: merupakan deskripsi buku
- Genres: merupakan genre buku
- Avg_Rating: mmerupakan rating rata-rata
- Num_Ratings: merupakan rating secara keseluruhan
- URL: merupakan link untuk mendownload buku

### **Book Variable**

Mari kita eksplorasi variabel book yang merupakan judul buku dalam dataset

In [15]:
books.Book.info()

<class 'pandas.core.series.Series'>
RangeIndex: 10000 entries, 0 to 9999
Series name: Book
Non-Null Count  Dtype 
--------------  ----- 
10000 non-null  object
dtypes: object(1)
memory usage: 78.3+ KB


### **Author Variable**

Eksplorasi variabel author, merupakan variabel yang menyimpan informasi mengenai nama-nama penulis buku

In [16]:
books.Author.info()

<class 'pandas.core.series.Series'>
RangeIndex: 10000 entries, 0 to 9999
Series name: Author
Non-Null Count  Dtype 
--------------  ----- 
10000 non-null  object
dtypes: object(1)
memory usage: 78.3+ KB


### **Description Variable**

Eksplorasi variabel description, yang menyimpan deskripsi/sinopsis dari masing-masing buku.

In [17]:
books.Description.info()

<class 'pandas.core.series.Series'>
RangeIndex: 10000 entries, 0 to 9999
Series name: Description
Non-Null Count  Dtype 
--------------  ----- 
9923 non-null   object
dtypes: object(1)
memory usage: 78.3+ KB


### **Genres Variable**

Explorasi variabel genres - merupakan variabel yang berisi data genre dari masing-masing buku


In [18]:
books.Genres.info()

<class 'pandas.core.series.Series'>
RangeIndex: 10000 entries, 0 to 9999
Series name: Genres
Non-Null Count  Dtype 
--------------  ----- 
10000 non-null  object
dtypes: object(1)
memory usage: 78.3+ KB


### **Avg_Rating Variable**

Eksplorasi variabel avg_rating - merupakan rating rata-rata dari masing-masing buku


In [19]:
books.Avg_Rating.info()

<class 'pandas.core.series.Series'>
RangeIndex: 10000 entries, 0 to 9999
Series name: Avg_Rating
Non-Null Count  Dtype  
--------------  -----  
10000 non-null  float64
dtypes: float64(1)
memory usage: 78.3 KB


### **Num_Ratings Variable**

Eksplorasi variabel num_rating - merupakan rating keseluruhan dari masing-masing buku


In [20]:
books.Num_Ratings.info()

<class 'pandas.core.series.Series'>
RangeIndex: 10000 entries, 0 to 9999
Series name: Num_Ratings
Non-Null Count  Dtype 
--------------  ----- 
10000 non-null  object
dtypes: object(1)
memory usage: 78.3+ KB


### **URL Variable**

Eksplorasi variabel url - merupakan link tempat untuk mengunduh buku

In [21]:
books.URL.info()

<class 'pandas.core.series.Series'>
RangeIndex: 10000 entries, 0 to 9999
Series name: URL
Non-Null Count  Dtype 
--------------  ----- 
10000 non-null  object
dtypes: object(1)
memory usage: 78.3+ KB


## **Data Preprocessing**

Dikarenakan data telah rapi, maka kita akan lanjut ke data preparation

## **Data Preparation**

### **Mengatasi Missing Value**

Mencari apakah di dalam data yang kita miliki terdapat missing value

In [22]:
books.isnull().sum()

,0
Book,0
Author,0
Description,77
Genres,0
Avg_Rating,0
Num_Ratings,0
URL,0


Berdasarkan hasil di atas, ternyata terdapat kolom dengan missing value, yaitu pada variabel Description sebanyak 77 buah. Jika demikian, kita akan menghapus semua data yang kosong tersebut.

In [23]:
books = books.dropna()

In [24]:
books.isnull().sum()

,0
Book,0
Author,0
Description,0
Genres,0
Avg_Rating,0
Num_Ratings,0
URL,0


#### **Membersihkan Data Genres**

In [25]:
fix_books = books
fix_books

,Book,Author,Description,Genres,Avg_Rating,Num_Ratings,URL
0,To Kill a Mockingbird,Harper Lee,The unforgettable novel of a childhood in a sl...,"['Classics', 'Fiction', 'Historical Fiction', ...",4.27,"5,691,311",https://www.goodreads.com/book/show/2657.To_Ki...
1,Harry Potter and the Philosopher’s Stone (Harr...,J.K. Rowling,Harry Potter thinks he is an ordinary boy - un...,"['Fantasy', 'Fiction', 'Young Adult', 'Magic',...",4.47,"9,278,135",https://www.goodreads.com/book/show/72193.Harr...
2,Pride and Prejudice,Jane Austen,"Since its immediate success in 1813, Pride and...","['Classics', 'Fiction', 'Romance', 'Historical...",4.28,"3,944,155",https://www.goodreads.com/book/show/1885.Pride...
3,The Diary of a Young Girl,Anne Frank,Discovered in the attic in which she spent the...,"['Classics', 'Nonfiction', 'History', 'Biograp...",4.18,"3,488,438",https://www.goodreads.com/book/show/48855.The_...
4,Animal Farm,George Orwell,Librarian's note: There is an Alternate Cover ...,"['Classics', 'Fiction', 'Dystopia', 'Fantasy',...",3.98,"3,575,172",https://www.goodreads.com/book/show/170448.Ani...
...,...,...,...,...,...,...,...
9995,"Breeders (Breeders Trilogy, #1)",Ashley Quigley,How far would you go? If human society was gen...,"['Dystopia', 'Science Fiction', 'Post Apocalyp...",3.44,276,https://www.goodreads.com/book/show/22085400-b...
9996,Dynamo,Eleanor Gustafson,Jeth Cavanaugh is searching for a new life alo...,[],4.23,60,https://www.goodreads.com/book/show/20862902-d...
9997,The Republic of Trees,Sam Taylor,This dark fable tells the story of four Englis...,"['Fiction', 'Horror', 'Dystopia', 'Coming Of A...",3.29,383,https://www.goodreads.com/book/show/891262.The...
9998,"Waking Up (Healing Hearts, #1)",Renee Dyer,For Adriana Monroe life couldn’t get any bette...,"['New Adult', 'Romance', 'Contemporary Romance...",4.13,263,https://www.goodreads.com/book/show/19347252-w...


In [26]:
fix_books['Genres'] = fix_books['Genres'].map(lambda x: x.lstrip('[').rstrip(']'))
fix_books['Genres'] = fix_books['Genres'].str.replace("'", "")
fix_books['Genres'] = fix_books['Genres'].str.split(',').str[:3].str.join(', ')
fix_books.head()


<ipython-input-26-1928548676>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fix_books['Genres'] = fix_books['Genres'].map(lambda x: x.lstrip('[').rstrip(']'))
<ipython-input-26-1928548676>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fix_books['Genres'] = fix_books['Genres'].str.replace("'", "")
<ipython-input-26-1928548676>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the doc

,Book,Author,Description,Genres,Avg_Rating,Num_Ratings,URL
0,To Kill a Mockingbird,Harper Lee,The unforgettable novel of a childhood in a sl...,"Classics, Fiction, Historical Fiction",4.27,"5,691,311",https://www.goodreads.com/book/show/2657.To_Ki...
1,Harry Potter and the Philosopher’s Stone (Harr...,J.K. Rowling,Harry Potter thinks he is an ordinary boy - un...,"Fantasy, Fiction, Young Adult",4.47,"9,278,135",https://www.goodreads.com/book/show/72193.Harr...
2,Pride and Prejudice,Jane Austen,"Since its immediate success in 1813, Pride and...","Classics, Fiction, Romance",4.28,"3,944,155",https://www.goodreads.com/book/show/1885.Pride...
3,The Diary of a Young Girl,Anne Frank,Discovered in the attic in which she spent the...,"Classics, Nonfiction, History",4.18,"3,488,438",https://www.goodreads.com/book/show/48855.The_...
4,Animal Farm,George Orwell,Librarian's note: There is an Alternate Cover ...,"Classics, Fiction, Dystopia",3.98,"3,575,172",https://www.goodreads.com/book/show/170448.Ani...


#### **Mengecek jumlah fix_books**


In [27]:
len(fix_books.Book.unique())

9795

### **Mengubah Data book, author, dan genres menjadi list**

In [28]:
book = fix_books['Book'].tolist()
author = fix_books['Author'].tolist()
genres = fix_books['Genres'].tolist()

In [29]:
print(len(book))
print(len(author))
print(len(genres))

9923
9923
9923


In [30]:
# Membuat dictionary untuk data ‘book’, ‘author’, dan ‘genres’
new_books = pd.DataFrame({
    'title': book,
    'author': author,
    'genre': genres
})
new_books

,title,author,genre
0,To Kill a Mockingbird,Harper Lee,"Classics, Fiction, Historical Fiction"
1,Harry Potter and the Philosopher’s Stone (Harr...,J.K. Rowling,"Fantasy, Fiction, Young Adult"
2,Pride and Prejudice,Jane Austen,"Classics, Fiction, Romance"
3,The Diary of a Young Girl,Anne Frank,"Classics, Nonfiction, History"
4,Animal Farm,George Orwell,"Classics, Fiction, Dystopia"
...,...,...,...
9918,"Breeders (Breeders Trilogy, #1)",Ashley Quigley,"Dystopia, Science Fiction, Post Apocalyptic"
9919,Dynamo,Eleanor Gustafson,
9920,The Republic of Trees,Sam Taylor,"Fiction, Horror, Dystopia"
9921,"Waking Up (Healing Hearts, #1)",Renee Dyer,"New Adult, Romance, Contemporary Romance"


### **Modelling**

Pada tahap modelling, saya membuat sebuah model machine learning dengan menggunakan algoritma **Content Based Filtering**

### **Content Based Filtering**

In [31]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Inisialisasi TfidfVectorizer
vector = TfidfVectorizer()

# Melakukan perhitungan idf pada data genre
vector.fit(new_books['genre'])

# Mapping array dari fitur index integer ke fitur nama
vector.get_feature_names_out()

array(['18th', '19th', '20th', '21st', 'about', 'abuse', 'action', 'adhd',
       'adoption', 'adult', 'adventure', 'africa', 'african', 'age',
       'agriculture', 'aliens', 'alternate', 'amazon', 'america',
       'american', 'amish', 'ancient', 'and', 'angels', 'animal',
       'animals', 'anthologies', 'anthropology', 'apocalyptic',
       'architecture', 'art', 'arthurian', 'artificial', 'arts',
       'asexual', 'asia', 'asian', 'astrology', 'astronomy', 'audiobook',
       'australia', 'autistic', 'autobiography', 'awards', 'bande',
       'banned', 'baseball', 'basketball', 'batman', 'bdsm', 'beast',
       'beauty', 'biography', 'biology', 'bizarro', 'book', 'books',
       'brazil', 'british', 'buddhism', 'buisness', 'business', 'canada',
       'catholic', 'cats', 'century', 'change', 'chapter', 'chick',
       'childrens', 'china', 'christian', 'christianity', 'christmas',
       'church', 'civil', 'classics', 'climate', 'club', 'collections',
       'college', 'comedy', '

In [32]:
matrix = vector.fit_transform(new_books['genre'])
matrix.shape

(9923, 390)

In [33]:
matrix.todense()

matrix([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.57201705, 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        ...,
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ]])

In [34]:
pd.DataFrame(
    matrix.todense(),
    columns=vector.get_feature_names_out(),
    index=new_books.title
).sample(22, axis=1).sample(10, axis=0)

,werewolves,space,zen,psychology,change,french,justice,stories,ukrainian,kids,...,dark,love,aliens,unicorns,anthologies,economics,shapeshifters,school,fantasy,holocaust
title,,,,,,,,,,,,,,,,,,,,,
Our Souls at Night,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
"Sackett's Land (The Sacketts, #1)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
Non Friction,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
Tell It to the World: International Justice and the Secret Campaign to Hide Mass Murder in Kosovo,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
It's All Too Much: An Easy Plan for Living a Richer Life with Less Stuff,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
The Pale Blue Eye,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
"Dom Wars: Round Four (Dom Wars, #4)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
The Last of the Just,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.810219
The Chimney Sweeper's Boy,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000


In [35]:
from sklearn.metrics.pairwise import cosine_similarity

kemiripan = cosine_similarity(matrix)
kemiripan

array([[1.        , 0.18613217, 0.54848941, ..., 0.1517404 , 0.        ,
        0.        ],
       [0.18613217, 1.        , 0.10698858, ..., 0.07541329, 0.17220613,
        0.        ],
       [0.54848941, 0.10698858, 1.        , ..., 0.08722023, 0.47693587,
        0.        ],
       ...,
       [0.1517404 , 0.07541329, 0.08722023, ..., 1.        , 0.        ,
        0.        ],
       [0.        , 0.17220613, 0.47693587, ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [36]:
data_kemiripan = pd.DataFrame(kemiripan, index=new_books['title'], columns=new_books['title'])
print('Shape:', data_kemiripan.shape)

data_kemiripan.sample(5, axis=1).sample(10, axis=0)

Shape: (9923, 9923)


title,The Bookstore Sisters,"The Lying Game (The Lying Game, #1)",The Woman in White,Bread and Jam for Frances,Mystic River
title,,,,,
"The Archon of Peace (Sovereign, #1)",0.000000,0.000000,0.000000,0.000000,0.000000
"Hiding Haelo (Candeon Heirs, #1)",0.000000,0.387435,0.000000,0.000000,0.000000
The Third Policeman,0.032083,0.000000,0.052606,0.029106,0.047898
حوار مع صديقي الملحد,0.000000,0.000000,0.000000,0.000000,0.000000
"The Scorch Trials (The Maze Runner, #2)",0.046932,0.380502,0.076953,0.042577,0.070066
(auster).mr.vertigo,0.045046,0.000000,0.073861,0.040866,0.067250
The Idea of One Religion,0.000000,0.000000,0.000000,0.000000,0.000000
Forever And Ever Amen,0.000000,0.000000,0.000000,0.000000,0.000000
On Earth As It Is in Heaven: Spiritual Attunement for Living Out God's Power Abiding in You,0.000000,0.000000,0.000000,0.000000,0.000000


#### **Mendapatkan Rekomendasi**

In [37]:
def books_recommendations(title, kemiripan=data_kemiripan, items=new_books[['title', 'genre']], k=5):


    # Mengambil data dengan menggunakan argpartition untuk melakukan partisi secara tidak langsung sepanjang sumbu yang diberikan
    # Dataframe diubah menjadi numpy
    # Range(start, stop, step)
    index = kemiripan.loc[:,title].to_numpy().argpartition(
        range(-1, -k, -1))

    # Mengambil data dengan similarity terbesar dari index yang ada
    closest = kemiripan.columns[index[-1:-(k+2):-1]]

    # Drop nama_movie agar nama movie yang dicari tidak muncul dalam daftar rekomendasi
    closest = closest.drop(title, errors='ignore')

    return pd.DataFrame(closest).merge(items).head(k)

In [38]:
new_books[new_books.title.eq('The Emigrants')]

,title,author,genre
7156,The Emigrants,W.G. Sebald,"Fiction, German Literature, Historical Fiction"


In [39]:
books_recommendations('The Emigrants')

,title,genre
0,Atemschaukel,"Fiction, Historical Fiction, German Literature"
1,Crabwalk,"Fiction, German Literature, Historical Fiction"
2,Chess Story,"Fiction, Classics, German Literature"
3,Heaven Has No Favorites,"Classics, Fiction, German Literature"
4,The Clown,"Fiction, German Literature, Classics"
